In [1]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium import webdriver as wd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests
import time
import re
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

### KBreport

In [2]:
url = "http://www.kbreport.com/history/pitcher/main"
driver = webdriver.Chrome("/Users/young/Documents/chromedriver")
driver.get(url)


In [20]:
#년도 설정
season_box = driver.find_element(By.CSS_SELECTOR,'#searchFrm > div.ltb-select-left > div.ltb-sl-1 > select.ltb-season-select')
season_box.send_keys('1991')


In [24]:
#검색
driver.find_element(By.CSS_SELECTOR,'#searchFrm > div.ltb-select-right > button').click()

In [25]:
#행 설정
row_box = driver.find_element(By.CSS_SELECTOR,'#resultListDiv > div.page-row-box > select')
row_box.send_keys('100')

In [44]:
html = driver.page_source
soup = BeautifulSoup(html,'html.parser')

paging = soup.select('#paging > a')
last_page = paging[-1]['href']
last_page = int(re.findall('[0-9]',last_page)[0])



In [ ]:
tf = soup.select('#resultListDiv > table > tbody > tr')


In [82]:
line = re.split('\n',tf[1].text)
del line[0],line[-1]
line

['1',
 '김성식',
 'LG',
 '0',
 '0',
 '2',
 '19',
 '1',
 '196.0',
 '1.65',
 '0.00',
 '0.00',
 '0.051',
 '-675.00',
 '0.00',
 '26.90',
 '2.66',
 '2.31',
 '12.08']

In [87]:
old_record = []
for i in range(1,len(tf)):
    line = re.split('\n',tf[i].text)
    del line[0],line[-1]
    old_record.append(line)

In [88]:
pd.DataFrame(old_record)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,1,김성식,LG,0,0,2,19,1,196.0,1.65,0.00,0.00,0.051,-675.00,0.00,26.90,2.66,2.31,12.08
1,2,선동열,해태,19,4,6,35,22,203.0,9.31,1.11,0.35,0.253,80.40,1.55,9.64,1.17,1.61,11.53
2,3,이강철,해태,15,11,3,36,24,214.7,8.09,2.26,0.84,0.248,70.40,3.19,4.38,2.68,3.00,6.23
3,4,김용수,LG,12,11,10,41,16,190.0,6.11,2.94,0.24,0.291,72.80,2.79,4.32,2.53,2.66,5.75
4,5,박동희,롯데,14,9,3,30,22,189.7,7.21,4.56,0.19,0.259,74.20,2.47,5.49,2.70,2.96,5.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,노상수,롯데,0,0,0,1,0,3.0,6.00,9.00,3.00,0.364,52.60,12.00,-0.28,9.51,9.68,-0.18
96,97,김건우,LG,1,1,0,2,2,5.0,3.60,9.00,1.80,0.250,52.10,9.00,-0.25,8.15,8.08,-0.18
97,98,김덕근,LG,1,1,0,11,3,42.0,3.43,5.79,0.43,0.238,74.80,3.00,0.55,4.80,4.68,-0.18
98,99,이상범,삼성,0,0,0,3,0,3.3,2.70,8.10,0.00,0.182,55.60,10.80,-0.22,8.55,8.38,-0.18


---

In [2]:
df_salary_old = pd.read_csv('/Users/young/Downloads/salary_baseball.csv',encoding='cp949')


### Statiz

In [172]:
for i in range(1,2):
    for i in range(1,10):
        s = i = 1
    print(s)

1


In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

wait = WebDriverWait(driver, 10)

wait.until(EC.element_to_be_selected(
    (By.CSS_SELECTOR, '#contents > div.sub-content')))


In [87]:
statiz_baseball_salary = pd.DataFrame(columns=["선수","연도","팀","연봉(만원)","WAR"])
for year in range(1997,2022):
    for team in [1,2,3,4,5,6,7,8,9,10,101,102]:
        url = f"http://www.statiz.co.kr/salary.php?opt=0&sopt={year}&cnv=&pos=&te={team}"
        year_salary_data = pd.read_html(url)[1]
        statiz_baseball_salary = pd.concat([statiz_baseball_salary,year_salary_data])

In [88]:
statiz_baseball_salary.to_csv('/Users/young/Desktop/statiz_salary.csv')

In [4]:
statiz_baseball_records = pd.DataFrame(columns=['순', '이름', '팀', 'WAR*', 'G', '타석', '타수', '득점', '안타', '2타', '3타', '홈런',
       '루타', '타점', '도루', '도실', '볼넷', '사구', '고4', '삼진', '병살', '희타', '희비', '타율',
       '출루', '장타', 'OPS', 'wOBA', 'wRC+', 'WAR*', 'WPA'])
year = 2021
url = f'http://www.statiz.co.kr/stat_at.php?opt=0&sopt=0&re=0&ys={year}&ye={year}&se=0&te=&tm=&ty=2016&qu=auto&po=51&as=&ae=&hi=&un=&pl=&da=1&o1=WAR_ALL_ADJ&o2=TPA&de=1&lr=0&tr=&cv=&ml=1&sn=30&si=&cn='
df = pd.read_html(url)[0]
df.columns = df.columns.droplevel(0)
statiz_baseball_records = pd.concat([statiz_baseball_records,df])


In [ ]:
statiz_baseball_records

### KBO 
---

In [25]:
import time
from selenium.webdriver.chrome.options import Options

In [69]:

url1 = "https://www.koreabaseball.com/Record/Player/PitcherBasic/Basic1.aspx?sort=ERA_RT"
url2 = "https://www.koreabaseball.com/Record/Player/PitcherBasic/Basic2.aspx?sort=ERA_RT"
url3 = "https://www.koreabaseball.com/Record/Player/PitcherBasic/Detail1.aspx"
url4 = "https://www.koreabaseball.com/Record/Player/PitcherBasic/Detail2.aspx"


def seleniums(url):

    options = Options()
    user_agent = "Mozilla/5.0 (Linux; Android 9; SM-G975F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.83 Mobile Safari/537.36"
    options.add_argument('user-agent=' + user_agent)

    driver = webdriver.Chrome("/Users/young/Documents/chromedriver")
    driver.get(url)
    
    game_box = driver.find_element(By.CSS_SELECTOR,'#cphContents_cphContents_cphContents_ddlSeries_ddlSeries')
    game_box.send_keys('KBO 정규시즌')

    record_outer = []
    
    for year in range(2002,2023):
        time.sleep(1)
        print(year)

        driver.implicitly_wait(10)
        season_box = driver.find_element(By.CSS_SELECTOR,'#cphContents_cphContents_cphContents_ddlSeason_ddlSeason')
        season_box.send_keys(year)
        time.sleep(1)
        
        path = '//*[@id="cphContents_cphContents_cphContents_udpContent"]/div[2]/div[1]/ul/li[2]/a'
        driver.find_element_by_xpath(path).send_keys('ENTER')
        driver.find_element_by_xpath(path).click()
        time.sleep(1)

        next_path = '//*[@id="cphContents_cphContents_cphContents_udpContent"]/div[2]/div[2]/a[2]'
        driver.find_element_by_xpath(next_path).click()
        time.sleep(1)

        html = driver.page_source
        soup = BeautifulSoup(html,'html.parser')
        tf = soup.select('#cphContents_cphContents_cphContents_udpContent > div.record_result > table > tbody > tr')

        for i in range(0,len(tf)):
            line = re.split('\n',tf[i].text)
            del line[0],line[-1]
            line.insert(0,year)
            record_outer.append(line)

        driver.implicitly_wait(10)
    

    return record_outer


In [ ]:
record_outer_1_1 = seleniums(url1)
record_outer_1_2 = seleniums(url2)
record_outer_2_1 = seleniums(url1)
record_outer_2_2 = seleniums(url1)

In [71]:
kb_1_1 = pd.DataFrame(record_outer_1_1)
kb_1_2 = pd.DataFrame(record_outer_1_2)
kb_2_1 = pd.DataFrame(record_outer_2_1)
kb_2_2 = pd.DataFrame(record_outer_2_2)


In [72]:
kb_1_1.to_csv('/Users/young/Desktop/kb_1_1.csv')
kb_1_2.to_csv('/Users/young/Desktop/kb_1_2.csv')
kb_2_1.to_csv('/Users/young/Desktop/kb_2_1.csv')
kb_2_2.to_csv('/Users/young/Desktop/kb_2_2.csv')


In [68]:
kb_1_1 = pd.read_csv('/Users/young/Desktop/kb_1_1.csv')
kb_1_2 = pd.read_csv('/Users/young/Desktop/kb_1_2.csv')
kb_2_1 = pd.read_csv('/Users/young/Desktop/kb_2_1.csv')
kb_2_2 = pd.read_csv('/Users/young/Desktop/kb_2_2.csv')

kb_1_1 = kb_1_1.iloc[:,1:]
kb_1_1_old = kb_1_1[kb_1_1['0']<2002]
kb_1_1_new = kb_1_1[kb_1_1['0']>=2002]
kb_1_1_new = kb_1_1_new.iloc[:,0:-2]
kb_1_2 = kb_1_2.iloc[:,1:]
kb_2_1 = kb_2_1.iloc[:,1:]
kb_2_2 = kb_2_2.iloc[:,1:]

col_kb_1_1_new = ['년도','순위','선수명','팀명','ERA','G','W','L','SV','HLD','WPCT','IP','H','HR','BB','HBP','SO','R','ER','WHIP']
col_kb_1_1_old = ['년도','순위','선수명','팀명','ERA','G','CG','SHO','W','L','SV','HLD','WPCT','TBF','IP','H','HR','BB','HBP','SO','R','ER']
col_kb_1_2 = ['년도','순위','선수명','팀명','ERA','CG','SHO','QS','BSV','TBF','NP','AVG','2B','3B','SAC','SF','IBB','WP','BK']
col_kb_2_1 = ['년도','순위','선수명','팀명','ERA','GS','Wgs','Wgr','GF','SVO','TS','GDP','GO','AO','GO/AO']
col_kb_2_2 = ['년도','순위','선수명','팀명','ERA','BABIP','P/G','P/IP','K/9','BB/9','K/BB','OBP','SLG','OPS']

kb_1_1_new.columns = col_kb_1_1_new
kb_1_1_old.columns = col_kb_1_1_old
kb_1_2.columns = col_kb_1_2
kb_2_1.columns = col_kb_2_1
kb_2_2.columns = col_kb_2_2

kb_1_1_new.reset_index(drop=True, inplace=True)

In [84]:
# kb_1_1_new와 kb_1_2의 5번째 열부터 끝까지를 merge하여 description 변수에 저장합니다.
for i in [kb_1_2.iloc[:,5:],kb_2_1.iloc[:,5:],kb_2_2.iloc[:,5:]]:
    kb_1_1_new = kb_1_1_new.merge(i, left_index=True, right_index=True)
kb_1_1_new

,년도,순위,선수명,팀명,ERA,G,W,L,SV,HLD,...,GO/AO,BABIP,P/G,P/IP,K/9,BB/9,K/BB,OBP,SLG,OPS
0,2002,1,엘비라,삼성,2.50,22,13,6,0,0,...,0.79,0.281,101.3,16.3,7.29,3.42,2.13,0.306,0.323,0.629
1,2002,2,송진우,한화,2.99,31,18,7,0,0,...,1.83,0.284,110.0,15.5,6.75,2.21,3.06,0.296,0.351,0.647
2,2002,3,임창용,삼성,3.08,36,17,6,2,0,...,0.97,0.278,86.2,15.2,7.05,1.50,4.71,0.296,0.396,0.692
3,2002,4,리오스,KIA,3.14,54,14,5,13,1,...,1.57,0.265,43.8,15.0,5.82,2.51,2.32,0.315,0.339,0.654
4,2002,5,이승호,SK,3.15,27,6,12,0,0,...,0.96,0.279,84.8,16.0,9.08,2.52,3.60,0.288,0.398,0.686
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,2022,18,박세웅,롯데,3.89,28,10,11,0,0,...,1.76,0.358,92.8,16.5,8.35,1.83,4.56,0.326,0.386,0.712
416,2022,19,원태인,삼성,3.92,27,10,8,0,0,...,0.96,0.314,98.9,16.1,7.08,2.07,3.42,0.312,0.386,0.698
417,2022,20,김민우,한화,4.36,29,6,11,0,0,...,0.86,0.274,94.9,16.9,7.12,4.64,1.54,0.336,0.386,0.722
418,2022,21,오원석,SSG,4.50,31,6,8,0,0,...,0.86,0.323,81.6,17.6,7.00,3.63,1.93,0.349,0.409,0.758


In [126]:
#팀명 맞추기
new_salary = statiz_baseball_salary[statiz_baseball_salary['연도']>2001]
new_salary['팀'] = new_salary['팀'].apply(lambda x: x.upper() if x == 'kt' else x)
kb_1_1_new['팀명'] = kb_1_1_new['팀명'].apply(lambda x: '히어로즈' if x == '우리' else x)
kb_1_1_new['팀명'] = kb_1_1_new['팀명'].apply(lambda x: '히어로즈' if x == '키움' else x)
kb_1_1_new['팀명'] = kb_1_1_new['팀명'].apply(lambda x: 'SK' if x == 'SSG' else x)

In [127]:
# A 리스트와 B 리스트에서 중복되지 않는 요소를 찾는 함수
def find_unique_elements(A, B):
    # A 리스트에서 B 리스트에 있는 요소를 제거한 새로운 리스트 생성
    unique_elements = [elem for elem in A if elem not in B]
    return unique_elements

print(find_unique_elements(kb_1_1_new['팀명'].unique(),new_salary['팀'].unique()))
print(find_unique_elements(new_salary['팀'].unique(),kb_1_1_new['팀명'].unique()))

[]
[]


In [140]:
new_data = kb_1_1_new.merge(new_salary,left_on=['년도','선수명','팀명'],right_on=['연도','선수','팀'])
new_data = new_data.drop(['연도','팀','선수','WAR'],axis=1)
new_data

,년도,순위,선수명,팀명,ERA,G,W,L,SV,HLD,...,BABIP,P/G,P/IP,K/9,BB/9,K/BB,OBP,SLG,OPS,연봉(만원)
0,2002,2,송진우,한화,2.99,31,18,7,0,0,...,0.284,110.0,15.5,6.75,2.21,3.06,0.296,0.351,0.647,13500
1,2002,3,임창용,삼성,3.08,36,17,6,2,0,...,0.278,86.2,15.2,7.05,1.50,4.71,0.296,0.396,0.692,30000
2,2002,5,이승호,SK,3.15,27,6,12,0,0,...,0.279,84.8,16.0,9.08,2.52,3.60,0.288,0.398,0.686,8000
3,2002,8,박명환,두산,3.44,30,14,10,0,0,...,0.311,96.8,15.6,8.19,3.30,2.49,0.321,0.356,0.677,10000
4,2002,9,손민한,롯데,3.67,23,4,9,0,0,...,0.282,91.7,14.8,5.19,1.90,2.73,0.307,0.438,0.745,12000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,2021,16,스트레일리,롯데,4.07,31,10,12,0,0,...,0.328,95.7,17.9,8.91,3.64,2.45,0.333,0.363,0.696,90000
286,2021,16,스트레일리,롯데,4.07,31,10,12,0,0,...,0.328,95.7,17.9,8.91,3.64,2.45,0.333,0.363,0.696,50000
287,2021,17,신민혁,NC,4.41,30,9,6,0,0,...,0.307,83.0,17.2,6.64,2.73,2.43,0.327,0.420,0.747,2700
288,2021,18,임기영,KIA,4.88,28,8,8,0,0,...,0.307,90.1,16.5,7.59,2.65,2.87,0.322,0.390,0.712,7000
